# Even reduction

In [1]:
using Pkg
pkg"add https://github.com/kalmarek/SymbolicWedderburn.jl#bl/nonperm"

import MutableArithmetics
const MA = MutableArithmetics
using MultivariatePolynomials
const MP = MultivariatePolynomials
using MultivariateBases
const MB = MultivariateBases

using DynamicPolynomials
@polyvar x

    Cloning git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating registry at `~/.julia/registries/General`
######################################################################## 100.0%
  Resolving package versions...
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Project.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/nonperm`
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Manifest.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/nonperm`


(x,)

We would like to find the minimum value of the polynomial

In [2]:
poly = x^4 - 2x^2

using SymbolicWedderburn
using PermutationGroups
using Cyclotomics
using SumOfSquares

This is still a work in progress in SumOfSquares, so we need to define things here that will be moved inside SumOfSquares.jl once SymbolicWedderburn.jl is released:

In [3]:
struct ScaledPerm2{T, I} <: AbstractPerm
    indices::Vector{Pair{I, T}}
end
Base.one(p::ScaledPerm2{T}) where {T} = ScaledPerm2([i => one(T) for i in eachindex(p.indices)])
Base.:(==)(p::ScaledPerm2, q::ScaledPerm2) = p.indices == q.indices
Base.hash(p::ScaledPerm2, u::UInt64) = hash(p.indices, u)
SymbolicWedderburn.degree(p::ScaledPerm2) = length(p.indices)
Base.:^(i::Integer, p::ScaledPerm2) = p.indices[i]
function SymbolicWedderburn.add_inverse_permutation!(result, val, i::Int, j::Pair)
    result[i, j.first] += val / j.second
end

function permutation(ehom::SymbolicWedderburn.ExtensionHomomorphism{T}, els::Vector{T}) where {T}
    return Perm([ehom[el] for el in els])
end
function permutation(ehom::SymbolicWedderburn.ExtensionHomomorphism{<:AbstractMonomial}, terms::Vector{<:AbstractTerm})
    return ScaledPerm2([ehom[monomial(term)] => coefficient(term) for term in terms])
end
function (ehom::SymbolicWedderburn.ExtensionHomomorphism)(action)
    return permutation(ehom, [f^action for f in ehom.features])
end

function SymbolicWedderburn.ExtensionHomomorphism(basis::MB.MonomialBasis)
    monos = collect(basis.monomials)
    mono_to_index = Dict(monos[i] => i for i in eachindex(monos))
    return SymbolicWedderburn.ExtensionHomomorphism(monos, mono_to_index)
end

function MP.polynomialtype(::Type{<:MB.AbstractPolynomialVectorBasis{PT}}, T::Type) where PT
    C = MP.coefficienttype(PT)
    U = MA.promote_operation(*, C, T)
    V = MA.promote_operation(+, U, U)
    return MP.polynomialtype(PT, V)
end

struct SymmetricIdeal{CT, GT} <: Certificate.AbstractIdealCertificate
    cone::CT
    group::GT
end
SumOfSquares.matrix_cone_type(::Type{<:SymmetricIdeal{CT}}) where {CT} = SumOfSquares.matrix_cone_type(CT)
Certificate.get(::Type{<:SymmetricIdeal}, ::SumOfSquares.Certificate.GramBasisType) = Vector{MB.FixedPolynomialBasis}
Certificate.zero_basis_type(::Type{<:SymmetricIdeal}) = MB.MonomialBasis
Certificate.zero_basis(::SymmetricIdeal) = MB.MonomialBasis
Certificate.get(::SymmetricIdeal, ::Certificate.ReducedPolynomial, poly, domain) = poly
function Certificate.get(cert::SymmetricIdeal, ::Certificate.GramBasis, poly)
    basis = Certificate.maxdegree_gram_basis(MB.MonomialBasis, MP.variables(poly), MP.maxdegree(poly))
    R = SymbolicWedderburn.symmetry_adapted_basis(Float64, cert.group, basis)
    return map(R) do Ri
        FixedPolynomialBasis(convert(Matrix{Float64}, Ri) * basis.monomials)
    end
end

We define the custom group as follows:

In [4]:
struct EvenOddAction <: GroupElem
    identity::Bool
end
PermutationGroups.order(a::EvenOddAction) = a.identity ? 1 : 2
Base.one(::EvenOddAction) = EvenOddAction(true)
Base.inv(a::EvenOddAction) = a
PermutationGroups.mul!(::EvenOddAction, a::EvenOddAction, b::EvenOddAction) = EvenOddAction(xor(a.identity, b.identity))
function Base.:^(mono::AbstractMonomial, a::EvenOddAction)
    if a.identity || iseven(MP.degree(mono))
        return 1 * mono
    else
        return -1 * mono
    end
end

struct EvenOddSymmetry <: Group
end
_orbit(cc) = PermutationGroups.Orbit(cc, Dict(a => nothing for a in cc))
SymbolicWedderburn.conjugacy_classes_orbit(::EvenOddSymmetry) = [_orbit([EvenOddAction(true)]), _orbit([EvenOddAction(false)])]

G = EvenOddSymmetry()

Main.##268.EvenOddSymmetry()

We can exploit the symmetry as follows:

In [5]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
con_ref = @constraint(model, poly - t in SOSCone(), ideal_certificate = SymmetricIdeal(SOSCone(), G))
optimize!(model)
@show value(t)
value(t)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.00e-01 Pobj: -4.4265675e+00 Ad: 9.00e-01 Dobj: -3.0000515e-02 
Iter:  2 Ap: 1.00e+00 Pobj: -4.9278250e+00 Ad: 9.00e-01 Dobj:  1.3441397e+00 
Iter:  3 Ap: 9.00e-01 Pobj: -1.7730372e+00 Ad: 9.00e-01 Dobj: -6.8418354e-01 
Iter:  4 Ap: 9.00e-01 Pobj: -1.1059483e+00 Ad: 9.00e-01 Dobj: -9.8702448e-01 
Iter:  5 Ap: 9.00e-01 Pobj: -1.0107592e+00 Ad: 9.00e-01 Dobj: -9.9870634e-01 
Iter:  6 Ap: 9.00e-01 Pobj: -1.0010797e+00 Ad: 1.00e+00 Dobj: -9.9999427e-01 
Iter:  7 Ap: 9.00e-01 Pobj: -1.0001071e+00 Ad: 1.00e+00 Dobj: -1.0000010e+00 
Iter:  8 Ap: 1.00e+00 Pobj: -1.0000113e+00 Ad: 1.00e+00 Dobj: -9.9999572e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -1.0000009e+00 Ad: 1.00e+00 Dobj: -1.0000016e+00 
Iter: 10 Ap: 1.00e+00 Pobj: -1.0000000e+00 Ad: 1.00e+00 Dobj: -1.0000000e+00 
value(t) = -1.0000000042454928


-1.0000000042454928

We indeed find `-1`, let's verify that symmetry was exploited:

In [6]:
gram_matrix(con_ref)

SparseGramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}(GramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}[GramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}([2.267148335865912e-9], FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}}(DynamicPolynomials.Polynomial{true,Float64}[x])), GramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}([0.999999999999832 -1.0000000011336343; -1.0000000011336343 1.0000000042456882], FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*